In [0]:
import os

os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-5741184051847129>, line 2
      1 import os
----> 2 os.environ["TF_ENABLE_ONEDNN_OPTS"] = 0

File <frozen os>:690, in __setitem__(self, key, value)

File <frozen os>:764, in encode(value)

TypeError: str expected, not int

#1 - Geração de Texto com Modelo Causal (Tucano-2b4)

📌 Resumo do Exercício

Este exercício demonstra como gerar texto com múltiplos prompts utilizando um modelo do tipo Causal Language Model (CLM), com execução paralela na GPU. Mostra como medir tempo de inferência e como o batch size impacta na performance.

💡 Aplicação

Geração de texto automático em português, a partir de instruções ou inícios de frase (prompts). Útil para geração criativa, completamento de respostas, storytelling e chatbots.

🔁 Input e Output

Input: Lista de frases iniciais (prompts) em português.
Output: Textos gerados pelo modelo, um para cada prompt.

In [0]:
import os
import time
import random
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# ========================
# Configurações iniciais
# ========================
# TODO: você pode alterar o modelo aqui para outro compatível do Hugging Face
MODEL_ID    = "TucanoBR/Tucano-2b4-Instruct"
CACHE_DIR   = os.path.expanduser("~/.cache/huggingface/transformers")
MAX_NEW     = 64                  # número de tokens a serem gerados
TEMPERATURE = 0.7                 # controla a aleatoriedade da geração
SEED        = 42
random.seed(SEED)
torch.manual_seed(SEED)

# ========================
# Carregamento do modelo
# ========================
# Usamos cache local para acelerar a reutilização
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, cache_dir=CACHE_DIR)
model     = AutoModelForCausalLM.from_pretrained(MODEL_ID, cache_dir=CACHE_DIR)
tokenizer.pad_token = tokenizer.eos_token  # evita warning no Hugging Face

# ========================
# Prompts de entrada
# ========================
# TODO: adicione outros prompts aqui para testar variações de saída
prompts = [
    "A XXV Escola Regional de Alto Desempenho da Região Sul (ERAD/RS 2025) acontecerá em Foz do Iguaçu. Durante o evento, os especialistas discutirão",
    "Os principais objetivos da ERAD/RS incluem qualificar profissionais em processamento de alto desempenho. No futuro, espera-se que",
    "Entre 23 e 25 de abril de 2025, participantes visitarão o Itaipu Parquetec, onde"
]

# ====================================================
# Função de geração de texto em lote com GPU
# ====================================================
@torch.inference_mode()  # Desativa o autograd para inferência mais leve
def generate_batch(texts):
    # Verifica se há GPU disponível
    if not torch.cuda.is_available():
        raise RuntimeError("GPU requerida mas não encontrada.")

    # Move o modelo para GPU — ativando paralelismo nos kernels da geração
    model.to("cuda", non_blocking=True)

    # Tokenização em lote (paralelismo ativado aqui)
    inputs = tokenizer(
        texts,
        return_tensors="pt",
        padding=True,              # garante alinhamento do batch
        padding_side='left'        # importante para modelos auto-regressivos
    ).to("cuda", non_blocking=True)

    # Sincroniza a GPU antes de medir tempo
    torch.cuda.synchronize()
    t0 = time.perf_counter()

    # Geração paralela em lote (CUDA executa múltiplas sequências simultaneamente)
    outs = model.generate(
        **inputs,
        max_new_tokens=MAX_NEW,
        do_sample=True,
        temperature=TEMPERATURE,
        top_k=50,
        top_p=0.9,
        repetition_penalty=1.15,
        no_repeat_ngram_size=2,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

    # Espera final da GPU antes de medir tempo total
    torch.cuda.synchronize()
    elapsed = time.perf_counter() - t0

    print(f"\nGeração concluída em {elapsed:.2f} segundos\n")

    # Decodificação e impressão
    for i, txt in enumerate(tokenizer.batch_decode(outs, skip_special_tokens=True), 1):
        print(f"---- Saída {i} ----\n{txt}\n")

# ========================
# Execução principal
# ========================
if __name__ == "__main__":
    # TODO: experimente rodar esta função com mais ou menos prompts
    generate_batch(prompts)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


Geração concluída em 3.73 segundos

---- Saída 1 ----
A XXV Escola Regional de Alto Desempenho da Região Sul (ERAD/RS 2025) acontecerá em Foz do Iguaçu. Durante o evento, os especialistas discutirão a relação entre a Ciência da Computação, as Tecnologias da Informação e Comunicação (TICs) e o Desenvolvimento Sustentável.
Professores e alunos das áreas de Ciências Exatas, Engenharias e Tecnologia, bem como representantes de instituições públicas e privadas, estão convidados para participar do encontro. Para participar, inscreva-se no

---- Saída 2 ----
Os principais objetivos da ERAD/RS incluem qualificar profissionais em processamento de alto desempenho. No futuro, espera-se que a escola seja um centro para estudantes e docentes de pós-graduação na área de processamento digital de sinais e processamento paralelo. Além disso, espera-se, também, uma forte colaboração com o Instituto Tecnológico e de Pesquisas do Estado do Rio Grande do Sul (ITP/RS) e a Universidade Federal do Pampa (Uni

---------------------------------------------------------------------------
HFValidationError                         Traceback (most recent call last)
File /databricks/python/lib/python3.12/site-packages/transformers/utils/hub.py:424, in cached_files(path_or_repo_id, filenames, cache_dir, force_download, resume_download, proxies, token, revision, local_files_only, subfolder, repo_type, user_agent, _raise_exceptions_for_gated_repo, _raise_exceptions_for_missing_entries, _raise_exceptions_for_connection_errors, _commit_hash, **deprecated_kwargs)
    422 if len(full_filenames) == 1:
    423     # This is slightly better for only 1 file
--> 424     hf_hub_download(
    425         path_or_repo_id,
    426         filenames[0],
    427         subfolder=None if len(subfolder) == 0 else subfolder,
    428         repo_type=repo_type,
    429         revision=revision,
    430         cache_dir=cache_dir,
    431         user_agent=user_agent,
    432         force_download=force_download,
 

In [0]:
import os
import time
import random
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import List, Dict

# --------------------------- Configuração do experimento ---------------------------
MODEL_ID   = "TucanoBR/Tucano-2b4-Instruct"    # Modelo causal da Hugging Face
CACHE_DIR  = os.path.expanduser("~/.cache/huggingface/transformers")
MAX_NEW    = 64                                # tokens a serem gerados
TEMPERATURE = 0.7                              # aleatoriedade da geração
SIZES      = [2**i for i in range(7)]          # tamanhos de lote: 1, 2, 4, 8, 16, 32, 64
SEED       = 42
random.seed(SEED); torch.manual_seed(SEED)

# ----------------------------------------------------------------------------
# Função para carregar o modelo e tokenizer
# ----------------------------------------------------------------------------
def load_model(model_id: str, cache_dir: str):
    tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)
    model = AutoModelForCausalLM.from_pretrained(model_id, cache_dir=cache_dir)
    tokenizer.pad_token = tokenizer.eos_token  # define token de padding necessário para batching
    return tokenizer, model

tokenizer, model = load_model(MODEL_ID, CACHE_DIR)

# ----------------------------------------------------------------------------
# Função que gera *n* prompts distintos
# ----------------------------------------------------------------------------
def make_prompts(n: int) -> List[str]:
    base = "A XXV Escola Regional de Alto Desempenho da Região Sul (ERAD/RS 2025) acontecerá em Foz do Iguaçu."
    return [f"{base} — example {i}" for i in range(n)]

# ----------------------------------------------------------------------------
# Execução sequencial na CPU (sem paralelismo entre prompts)
# ----------------------------------------------------------------------------
@torch.inference_mode()
def run_cpu_seq(prompts: List[str]) -> float:
    """Gera texto um por um na CPU. Medimos o tempo total."""
    model.to("cpu")
    t0 = time.perf_counter()
    for p in prompts:
        inputs = tokenizer(p, return_tensors="pt", padding=True).to("cpu")
        _ = model.generate(
            **inputs,
            max_new_tokens=MAX_NEW,
            do_sample=True, temperature=TEMPERATURE,
            top_k=50, top_p=0.9,
            repetition_penalty=1.15,
            no_repeat_ngram_size=2,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    return time.perf_counter() - t0

# ----------------------------------------------------------------------------
# Execução em lote na GPU (ativando paralelismo verdadeiro)
# ----------------------------------------------------------------------------
@torch.inference_mode()
def run_gpu_batch(prompts: List[str]) -> float | None:
    """Gera todos os prompts em batch na GPU. Retorna o tempo total."""
    if not torch.cuda.is_available():
        return None

    try:
        model.to("cuda", non_blocking=True)

        # Aqui ativamos o paralelismo de entrada (tokenização em lote)
        inputs = tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True
        ).to("cuda", non_blocking=True)

        torch.cuda.synchronize()  # garante início de medição após alocação
        t0 = time.perf_counter()

        # O paralelismo é totalmente explorado aqui:
        outs = model.generate(
            **inputs,
            max_new_tokens=MAX_NEW,
            do_sample=True, temperature=TEMPERATURE,
            top_k=50, top_p=0.9,
            repetition_penalty=1.15,
            no_repeat_ngram_size=2,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

        torch.cuda.synchronize()  # espera conclusão total da geração
        return time.perf_counter() - t0

    finally:
        # Limpa memória GPU após a execução
        del inputs
        torch.cuda.empty_cache()

# ----------------------------------------------------------------------------
# Função principal de benchmark: compara CPU sequencial vs. GPU em lote
# ----------------------------------------------------------------------------
def benchmark() -> Dict[int, Dict[str, float | None]]:
    results: Dict[int, Dict[str, float | None]] = {}
    print(f"\n{'N':>5} | {'CPU-SEQ (s)':>11} | {'GPU (s)':>8} | Speed-up")
    print("-" * 43)

    for n in SIZES:
        prompts = make_prompts(n)

        t_cpu = run_cpu_seq(prompts)
        t_gpu = run_gpu_batch(prompts)

        speed = (t_cpu / t_gpu) if (t_gpu and t_gpu > 0) else None
        results[n] = {"cpu": t_cpu, "gpu": t_gpu, "speedup": speed}

        gpu_txt = f"{t_gpu:8.2f}" if t_gpu else "   N/A  "
        spd_txt = f"{speed:6.2f}×" if speed else "  —"
        print(f"{n:5d} | {t_cpu:11.2f} | {gpu_txt} | {spd_txt}")

    return results

# ----------------------------------------------------------------------------
# Execução principal: inicializa o benchmark e imprime a tabela comparativa
# ----------------------------------------------------------------------------
if __name__ == "__main__":
    benchmark()

2025-04-25 10:35:59.278525: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-25 10:35:59.294746: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-25 10:35:59.314821: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-25 10:35:59.320850: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-25 10:35:59.335098: I tensorflow/core/platform/cpu_feature_guar

tokenizer_config.json:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]


    N | CPU-SEQ (s) |  GPU (s) | Speed-up
-------------------------------------------
    1 |        7.73 |     3.65 |   2.12×
    2 |       15.17 |     3.59 |   4.23×
    4 |       30.34 |     3.82 |   7.95×
    8 |       57.74 |     4.57 |  12.64×


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

#2 - Inferência para Análise de Sentimentos em Português

📌 Resumo do Exercício

Este exercício demonstra como realizar inferência paralela (batch) em GPU utilizando um modelo pré-treinado em português (pysentimiento/bertweet-pt-sentiment) para classificação binária de sentimento (positivo ou negativo). A atividade mostra como medir o tempo de execução e comparar os ganhos de performance obtidos com uso de paralelismo na GPU, em contraste com execução sequencial na CPU.

💡 Aplicação

Classificação automática do sentimento em textos curtos, como comentários, avaliações ou tweets. Útil para análise de redes sociais, pesquisas de opinião, monitoramento de eventos e outras aplicações de NLP em português.

🔁 Input e Output

Input: Lista de frases em português, curtas e de opinião.
Output: Para cada frase, o modelo retorna:
- O rótulo de sentimento (positivo ou negativo)
- O score de confiança (probabilidade associada à predição)

In [0]:
import os
import time
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline

# ------------------------------------------------------------------------------
# CONFIGURAÇÃO DO MODELO E DADOS
# ------------------------------------------------------------------------------
MODEL_ID  = "pysentimiento/bertweet-pt-sentiment"
CACHE_DIR = os.path.expanduser("~/.cache/huggingface/transformers")

# TODO: você pode alterar, remover ou adicionar frases de teste aqui
SENTENCES = [
    "Estou muito animado para participar da XXV ERAD/RS 2025 em Foz do Iguaçu!",
    "Infelizmente perdi o prazo de submissão de trabalhos para a ERAD, fiquei desapontado.",
    "O cronograma de minicursos sobre computação exaescala parece excelente.",
    "A prorrogação das inscrições de novo foi frustrante para alguns participantes.",
    "O Fórum de Iniciação Científica da ERAD/RS costuma ser inspirador e motivador.",
]

# ------------------------------------------------------------------------------
# FUNÇÃO PARA CARREGAR O PIPELINE DE SENTIMENTO
# ------------------------------------------------------------------------------
def load_pipeline(model_id: str, cache_dir: str) -> TextClassificationPipeline:
    """
    Carrega tokenizer e modelo diretamente para a GPU (caso disponível).
    A aceleração é ativada via `device=0`, indicando uso de CUDA.
    """
    device = 0 if torch.cuda.is_available() else -1  # -1 = CPU
    tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)

    # O uso de float16 reduz uso de memória e acelera operações em GPU Ampere+
    model = AutoModelForSequenceClassification.from_pretrained(
        model_id,
        cache_dir=cache_dir,
        torch_dtype=torch.float16 if torch.cuda.is_available() else None
    )

    return TextClassificationPipeline(model=model, tokenizer=tokenizer, device=device)

# ------------------------------------------------------------------------------
# FUNÇÃO DE INFERÊNCIA COM MEDIÇÃO DE TEMPO
# ------------------------------------------------------------------------------
def run_gpu_batch(sentences: list[str]) -> None:
    """
    Realiza inferência paralela na GPU com todas as frases de uma vez (batch_size=n).
    Mede tempo total e imprime rótulos e scores.
    """
    pipe = load_pipeline(MODEL_ID, CACHE_DIR)

    torch.cuda.synchronize()  # garante que tudo esteja ocioso antes da medição
    t0 = time.perf_counter()

    # Aqui ocorre a inferência paralela real — todas as frases são processadas
    # em paralelo no Tensor Core da GPU, com otimização por lote
    outputs = pipe(sentences, batch_size=len(sentences), truncation=True)

    torch.cuda.synchronize()  # espera todos os kernels CUDA terminarem
    elapsed = time.perf_counter() - t0

    print(f"\nInferência em GPU concluída em {elapsed:.2f} segundos.\n")

    # Impressão dos resultados
    for s, out in zip(sentences, outputs):
        lbl = "positivo" if out["label"] == "POS" else "negativo"
        print(f"Frase: {s}\nSentimento: {lbl} (confiança {out['score']:.2f})\n")

# ------------------------------------------------------------------------------
# EXECUÇÃO PRINCIPAL
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    run_gpu_batch(SENTENCES)


tokenizer_config.json:   0%|          | 0.00/562 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/799k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/952 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

Device set to use cuda:0



Inferência em GPU concluída em 0.07 segundos.

Frase: Estou muito animado para participar da XXV ERAD/RS 2025 em Foz do Iguaçu!
Sentimento: positivo (confiança 0.95)

Frase: Infelizmente perdi o prazo de submissão de trabalhos para a ERAD, fiquei desapontado.
Sentimento: negativo (confiança 0.99)

Frase: O cronograma de minicursos sobre computação exaescala parece excelente.
Sentimento: positivo (confiança 0.98)

Frase: A prorrogação das inscrições de novo foi frustrante para alguns participantes.
Sentimento: negativo (confiança 0.95)

Frase: O Fórum de Iniciação Científica da ERAD/RS costuma ser inspirador e motivador.
Sentimento: positivo (confiança 0.95)



com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:434)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:473)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:737)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:510)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:616)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:643)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:49)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:293)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(Attr

In [0]:
import os
import time
import random
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from typing import List, Dict

# --------------------------- Configurações Globais ---------------------------
MODEL_ID  = "pysentimiento/bertweet-pt-sentiment"
CACHE_DIR = os.path.expanduser("~/.cache/huggingface/transformers")
SIZES     = [2**i for i in range(13)]
SEED      = 42
random.seed(SEED)

# --------------------------------------------------------------------
def load_model(model_id: str, cache_dir: str):
    """Carrega modelo e tokenizer do Hugging Face, com cache local."""
    tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)
    model = AutoModelForSequenceClassification.from_pretrained(model_id, cache_dir=cache_dir)
    return tokenizer, model

tokenizer, model = load_model(MODEL_ID, CACHE_DIR)
model.eval()  # Desativa dropout e outras variações estocásticas

# --------------------------------------------------------------------
def make_sentences(n: int) -> List[str]:
    """Gera *n* frases distintas simulando avaliações de evento."""
    templates = [
        "Estou muito animado para participar da XXV ERAD/RS 2025 em Foz do Iguaçu! - Exemplo {}",
        "Infelizmente perdi o prazo de submissão de trabalhos para a ERAD, fiquei desapontado. - Exemplo {}",
        "O cronograma de minicursos sobre computação exaescala parece excelente. - Exemplo {}",
        "A prorrogação das inscrições de novo foi frustrante para alguns participantes. - Exemplo {}",
        "O Fórum de Iniciação Científica da ERAD/RS costuma ser inspirador e motivador. - Exemplo {}"
    ]
    return [random.choice(templates).format(i) for i in range(n)]

# --------------------------------------------------------------------
@torch.inference_mode()
def run_cpu_seq(sentences: List[str]) -> float:
    """
    Executa a inferência uma por uma na CPU.
    Não há paralelismo entre exemplos.
    """
    model.to("cpu")
    t0 = time.perf_counter()
    for s in sentences:
        inputs = tokenizer(s, return_tensors="pt", truncation=True, padding=True).to("cpu")
        _ = model(**inputs)
    return time.perf_counter() - t0

# --------------------------------------------------------------------
@torch.inference_mode()
def run_gpu_batch(sentences: List[str]) -> float | None:
    """
    Executa inferência em lote na GPU.
    Aqui ocorre paralelismo verdadeiro: o modelo processa todas as entradas simultaneamente
    usando os núcleos CUDA da GPU, com alto aproveitamento computacional.
    """
    if not torch.cuda.is_available():
        return None

    try:
        model.to("cuda", non_blocking=True)

        # Tokenização em lote com padding para alinhamento
        inputs = tokenizer(
            sentences,
            return_tensors="pt",
            padding=True,
            truncation=True
        ).to("cuda", non_blocking=True)

        torch.cuda.synchronize()  # Garante que nada esteja pendente
        t0 = time.perf_counter()

        # Aqui o paralelismo entra em ação: todos os inputs são processados juntos
        _ = model(**inputs)

        torch.cuda.synchronize()  # Espera o fim da execução dos kernels
        return time.perf_counter() - t0
    finally:
        del inputs
        torch.cuda.empty_cache()

# --------------------------------------------------------------------
def benchmark() -> Dict[int, Dict[str, float | None]]:
    """
    Loop principal do benchmark.
    Para cada tamanho de batch definido em SIZES:
    - Gera frases de entrada
    - Executa em CPU e GPU
    - Registra e imprime tempo + aceleração relativa
    """
    results: Dict[int, Dict[str, float | None]] = {}
    print(f"\n{'N':>5} | {'CPU-SEQ (s)':>11} | {'GPU (s)':>8} | Speed-up")
    print("-" * 43)

    for n in SIZES:
        samples = make_sentences(n)

        t_cpu = run_cpu_seq(samples)
        t_gpu = run_gpu_batch(samples)

        speed = (t_cpu / t_gpu) if (t_gpu and t_gpu > 0) else None
        results[n] = {"cpu": t_cpu, "gpu": t_gpu, "speedup": speed}

        gpu_txt = f"{t_gpu:8.2f}" if t_gpu else "   N/A  "
        spd_txt = f"{speed:6.2f}×" if speed else "  —"
        print(f"{n:5d} | {t_cpu:11.2f} | {gpu_txt} | {spd_txt}")

    return results

# --------------------------------------------------------------------
if __name__ == "__main__":
    benchmark()



    N | CPU-SEQ (s) |  GPU (s) | Speed-up
-------------------------------------------
    1 |        0.03 |     0.01 |   3.97×
    2 |        0.05 |     0.01 |   5.10×
    4 |        0.11 |     0.01 |   8.70×
    8 |        0.19 |     0.02 |  10.58×
   16 |        0.29 |     0.03 |   9.37×
   32 |        0.58 |     0.03 |  17.10×
   64 |        1.14 |     0.06 |  20.60×
  128 |        2.39 |     0.20 |  11.80×
  256 |        4.75 |     0.31 |  15.47×
  512 |        9.23 |     0.63 |  14.62×
 1024 |       18.80 |     1.26 |  14.89×
 2048 |       38.63 |     2.33 |  16.59×
 4096 |       75.33 |     4.79 |  15.72×


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

#3 - Sumarização de Texto com Modelo PT-T5

📌 Resumo do Exercício

Este exercício demonstra como utilizar modelos de sequência para sequência (seq2seq) para gerar resumos automáticos de textos longos em português, com foco em eficiência. São aplicadas técnicas de inferência em lote (batch inference) usando GPU, aproveitando o paralelismo de hardware para acelerar a tarefa.

💡 Aplicação

A sumarização automática tem ampla aplicação em:
- Geração de resumos de notícias, artigos acadêmicos ou documentos corporativos.
- Assistentes virtuais que oferecem resumos rápidos de conteúdos longos.
- Ferramentas de produtividade com foco em leitura rápida ou extração de informações.

🔁 Input e Output

- Input: Lista de textos longos (strings em português) — cada um com cerca de 3 a 6 frases.
- Output: Um resumo automático para cada entrada, com no máximo 64 tokens gerados.

In [0]:
from __future__ import annotations
import os, time, torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# ------------------------------------------------------------------------------
# CONFIGURAÇÃO GERAL
# ------------------------------------------------------------------------------
MODEL_ID  = "recogna-nlp/ptt5-base-summ-cstnews"
CACHE_DIR = os.path.expanduser("~/.cache/huggingface/transformers")
MAX_NEW   = 64  # Máximo de tokens gerados por resumo

# Exemplos reais adaptados do site da ERAD/RS 2025
TEXTOS_LONGOS = [
    (
        "A XXV Escola Regional de Alto Desempenho da Região Sul (ERAD/RS 2025) "
        "ocorrerá entre 23 e 25 de abril em Foz do Iguaçu, Paraná. O evento "
        "comemora 25 anos promovendo cursos, palestras e desafios práticos sobre "
        "computação de alto desempenho, inteligência artificial e sistemas "
        "paralelos. Nesta edição, a programação inclui visita técnica à Itaipu "
        "Parquetec e painéis dedicados a exaescala e sustentabilidade energética."
    ),
    (
        "O Comitê de Programa da ERAD/RS convida estudantes e pesquisadores a "
        "submeterem trabalhos para o Fórum de Iniciação Científica. Serão aceitos "
        "artigos de até oito páginas descrevendo resultados parciais ou finais "
        "em processamento de alto desempenho, arquitetura de GPUs, compiladores "
        "e aplicações científicas. Os autores dos melhores artigos ganharão "
        "inscrição gratuita e premiação em dinheiro."
    ),
    (
        "Além dos minicursos tradicionais, a organização oferecerá um hackathon "
        "sobre otimização de kernels CUDA, com premiação para a equipe que "
        "alcançar maior aceleração no benchmark de simulação climática. Os "
        "participantes terão acesso a clusters com placas NVIDIA A100 e suporte "
        "de mentores da indústria."
    ),
]

# ------------------------------------------------------------------------------
# FUNÇÃO DE CARREGAMENTO DO MODELO
# ------------------------------------------------------------------------------
def carregar_modelo(model_id: str):
    """
    Carrega tokenizer e modelo para sumarização.
    O modelo é movido automaticamente para a GPU (device_map="auto")
    e usa precisão reduzida (float16) para desempenho otimizado.
    """
    if not torch.cuda.is_available():
        raise RuntimeError("🚫 CUDA não encontrada — a versão GPU exige placa NVIDIA.")

    tok = AutoTokenizer.from_pretrained(model_id, cache_dir=CACHE_DIR)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        model_id,
        cache_dir=CACHE_DIR,
        torch_dtype=torch.float16,
        device_map="auto"  # Distribui o modelo automaticamente na GPU disponível
    )
    return tok, model

# ------------------------------------------------------------------------------
# FUNÇÃO PRINCIPAL DE INFERÊNCIA EM LOTE
# ------------------------------------------------------------------------------
@torch.inference_mode()
def resumir_batch(textos: list[str]) -> None:
    """
    Realiza sumarização de todos os textos de uma vez,
    aproveitando o paralelismo da GPU via inferência batelada (batch).
    """
    tok, model = carregar_modelo(MODEL_ID)

    # Tokenização com padding e truncamento para que todas as entradas
    # fiquem no mesmo tamanho e possam ser processadas em lote (paralelo)
    inputs = tok(textos, return_tensors="pt", padding=True,
                 truncation=True, max_length=512).to("cuda", non_blocking=True)

    torch.cuda.synchronize()  # Garante que GPU esteja ociosa antes da medição
    t0 = time.perf_counter()

    # A geração ocorre com num_beams > 1 (beam search), ativando heurística de qualidade
    # O paralelismo ocorre aqui: todos os textos são resumidos em uma única chamada de GPU
    outs = model.generate(
        **inputs,
        max_new_tokens=MAX_NEW,
        num_beams=4,
        length_penalty=1.0,
        early_stopping=True
    )

    torch.cuda.synchronize()
    elapsed = time.perf_counter() - t0

    print(f"\n=== PT-T5 • sumarização batelada (GPU) — {elapsed:.2f} s ===\n")

    # Impressão dos resultados
    for idx, (orig, out_ids) in enumerate(zip(textos, outs), 1):
        resumo = tok.decode(out_ids, skip_special_tokens=True)
        print(f"── Texto {idx} (origem ERAD/RS) ──\n{orig}\n")
        print(f"▸ Resumo:\n{resumo}\n")

# ------------------------------------------------------------------------------
# PONTO DE ENTRADA
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    resumir_batch(TEXTOS_LONGOS)

tokenizer_config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/669 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/756k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Tensor Parallel requires torch.distributed to be initialized first.


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]


=== PT-T5 • sumarização batelada (GPU) — 0.60 s ===

── Texto 1 (origem ERAD/RS) ──
A XXV Escola Regional de Alto Desempenho da Região Sul (ERAD/RS 2025) ocorrerá entre 23 e 25 de abril em Foz do Iguaçu, Paraná. O evento comemora 25 anos promovendo cursos, palestras e desafios práticos sobre computação de alto desempenho, inteligência artificial e sistemas paralelos. Nesta edição, a programação inclui visita técnica à Itaipu Parquetec e painéis dedicados a exaescala e sustentabilidade energética.

▸ Resumo:
A XXV Escola Regional de Alto Desempenho da Região Sul ocorrerá entre 23 e 25 de abril em Foz do Iguaçu.

── Texto 2 (origem ERAD/RS) ──
O Comitê de Programa da ERAD/RS convida estudantes e pesquisadores a submeterem trabalhos para o Fórum de Iniciação Científica. Serão aceitos artigos de até oito páginas descrevendo resultados parciais ou finais em processamento de alto desempenho, arquitetura de GPUs, compiladores e aplicações científicas. Os autores dos melhores artigos ganharão

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:434)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:473)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:737)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:510)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:616)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:643)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:49)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:293)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(Attr

In [0]:
from __future__ import annotations
import os, time, random
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from typing import List, Dict

# ------------------------------------------------------------------------------
# CONFIGURAÇÃO GLOBAL DO EXPERIMENTO
# ------------------------------------------------------------------------------
MODEL_ID   = "recogna-nlp/ptt5-base-summ-cstnews"
CACHE_DIR  = os.path.expanduser("~/.cache/huggingface/transformers")
MAX_NEW    = 64                          # Limite de tokens a serem gerados
SIZES      = [2**i for i in range(8)]
SEED       = 42
random.seed(SEED)

# ------------------------------------------------------------------------------
def load_model(model_id: str, cache_dir: str):
    """Carrega o modelo e tokenizer do Hugging Face com cache local."""
    tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_id, cache_dir=cache_dir)
    return tokenizer, model

tokenizer, model = load_model(MODEL_ID, CACHE_DIR)
model.eval()

# ------------------------------------------------------------------------------
def make_articles(n: int) -> List[str]:
    """
    Gera *n* artigos longos para sumarização.
    A entrada usa o prefixo 'summarize:' pois T5 espera instruções explícitas.
    """
    base = (
        "A XXV Escola Regional de Alto Desempenho da Região Sul (ERAD/RS 2025) "
        "ocorrerá entre 23 e 25 de abril em Foz do Iguaçu, Paraná. O evento "
        "comemora 25 anos promovendo cursos, palestras e desafios práticos sobre "
        "computação de alto desempenho, inteligência artificial e sistemas "
        "paralelos. Nesta edição, a programação inclui visita técnica à Itaipu "
        "Parquetec e painéis dedicados a exaescala e sustentabilidade energética."
    )
    return [f"summarize: {base} Sample number {i}." for i in range(n)]

# ------------------------------------------------------------------------------
@torch.inference_mode()
def run_cpu_seq(inputs: List[str]) -> float:
    """
    Sumariza os textos um a um usando CPU.
    Não há paralelismo entre inputs, resultando em maior tempo total.
    """
    model.to("cpu")
    t0 = time.perf_counter()
    for text in inputs:
        tokens = tokenizer(text, return_tensors="pt", padding=True, max_length=512, truncation=True).to("cpu")
        _ = model.generate(
            **tokens,
            max_new_tokens=MAX_NEW,
            do_sample=False,
            num_beams=4,
            length_penalty=2.0,
            early_stopping=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    return time.perf_counter() - t0

# ------------------------------------------------------------------------------
@torch.inference_mode()
def run_gpu_batch(inputs: List[str]) -> float | None:
    """
    Sumariza todo o lote de textos simultaneamente usando GPU.
    Aqui é ativado o paralelismo total — múltiplas sequências são processadas
    de forma vetorizada usando os núcleos CUDA da GPU.
    """
    if not torch.cuda.is_available():
        return None

    try:
        model.to("cuda", non_blocking=True)
        tokens = tokenizer(inputs, return_tensors="pt", padding=True, max_length=512, truncation=True).to("cuda", non_blocking=True)

        torch.cuda.synchronize()  # garante que a GPU está ociosa antes de medir
        t0 = time.perf_counter()

        _ = model.generate(
            **tokens,
            max_new_tokens=MAX_NEW,
            do_sample=False,
            num_beams=4,
            length_penalty=2.0,
            early_stopping=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

        torch.cuda.synchronize()  # aguarda todos os kernels terminarem
        return time.perf_counter() - t0

    finally:
        del tokens
        torch.cuda.empty_cache()

# ------------------------------------------------------------------------------
def benchmark() -> Dict[int, Dict[str, float | None]]:
    """
    Loop principal de benchmark:
    - Executa sumarização para diferentes tamanhos de lote
    - Compara execução sequencial na CPU vs. paralela na GPU
    - Exibe uma tabela com tempos absolutos e speed-up relativo
    """
    results: Dict[int, Dict[str, float | None]] = {}
    print(f"\n{'N':>5} | {'CPU-SEQ (s)':>11} | {'GPU (s)':>8} | Speed-up")
    print("-" * 43)

    for n in SIZES:
        prompts = make_articles(n)

        t_cpu = run_cpu_seq(prompts)
        t_gpu = run_gpu_batch(prompts)

        speed = (t_cpu / t_gpu) if (t_gpu and t_gpu > 0) else None
        results[n] = {"cpu": t_cpu, "gpu": t_gpu, "speedup": speed}

        gpu_txt = f"{t_gpu:8.2f}" if t_gpu else "   N/A  "
        spd_txt = f"{speed:6.2f}×" if speed else "  —"
        print(f"{n:5d} | {t_cpu:11.2f} | {gpu_txt} | {spd_txt}")

    return results

# ------------------------------------------------------------------------------
if __name__ == "__main__":
    benchmark()

tokenizer_config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/669 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/756k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]


    N | CPU-SEQ (s) |  GPU (s) | Speed-up
-------------------------------------------
    1 |        0.86 |     0.55 |   1.57×
    2 |        2.23 |     0.86 |   2.58×
    4 |        4.60 |     0.97 |   4.72×
    8 |        7.88 |     1.08 |   7.27×
   16 |       14.15 |     1.39 |  10.17×
   32 |       27.34 |     2.29 |  11.91×
   64 |       52.99 |     4.29 |  12.36×
  128 |      106.61 |     8.43 |  12.65×


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

#4 - Tradução com mT5: Inglês para Português

📌 Resumo do Exercício

Este exercício realiza a tradução automática de frases em inglês para o português usando um modelo mT5-base especializado no par linguístico en→pt. A execução em lote na GPU demonstra como tarefas de NLP com modelos grandes (3B parâmetros) podem ser aceleradas com paralelismo de hardware.

💡 Aplicação

Aplicações práticas incluem:
- Tradução automática em sistemas de atendimento multilíngue.
- Ferramentas de localização e globalização de conteúdo.
- Sistemas de recomendação e resumo de conteúdo internacional.

🔁 Input e Output

- Input: Lista de frases em inglês (strings).
- Output: Lista de traduções para o português (strings geradas pelo modelo).

In [0]:
from __future__ import annotations
import os, time, torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# ------------------------------------------------------------------------------
# CONFIGURAÇÃO GLOBAL
# ------------------------------------------------------------------------------
MODEL_ID  = "unicamp-dl/mt5-3B-mmarco-en-pt"  # Modelo mT5 treinado para EN→PT
CACHE_DIR = os.path.expanduser("~/.cache/huggingface/transformers")
MAX_NEW   = 96  # Tokens máximos a serem gerados por tradução

# Frases de entrada (retiradas ou inspiradas do site da ERAD/RS 2025)
EN_SENTENCES = [
    "The XXV ERAD/RS 2025 will be held in Foz do Iguacu between April 23 and 25.",
    "Participants can visit the Itaipu Parquetec facilities during the event.",
    "The program features advanced courses on GPU programming and exascale computing.",
    "The deadline for paper submissions to the Scientific Initiation Forum was extended again.",
    "Winners of the CUDA kernel hackathon will receive cash prizes and free registration."
]

# ------------------------------------------------------------------------------
# FUNÇÃO DE CARREGAMENTO DO MODELO
# ------------------------------------------------------------------------------
def load_pipeline(model_id: str):
    """
    Carrega modelo e tokenizer diretamente na GPU, com uso de float16 (meia precisão).
    Isso reduz uso de VRAM e acelera as operações em hardwares modernos como T4, A10, A100.
    """
    if not torch.cuda.is_available():
        raise RuntimeError("🚫 Esta versão exige GPU CUDA — nenhuma placa detectada.")

    tok = AutoTokenizer.from_pretrained(model_id, cache_dir=CACHE_DIR)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        model_id,
        cache_dir=CACHE_DIR,
        torch_dtype=torch.float16,  # Ativa FP16 (meia precisão)
        device_map="auto"           # Coloca o modelo inteiro na GPU disponível
    ).to("cuda")

    return tok, model

# ------------------------------------------------------------------------------
# FUNÇÃO DE INFERÊNCIA EM LOTE
# ------------------------------------------------------------------------------
@torch.inference_mode()
def translate_batch(sentences: list[str]) -> None:
    """
    Executa tradução de todas as frases em lote.
    Isso ativa o paralelismo da GPU — todas as entradas são processadas simultaneamente.
    """
    tok, model = load_pipeline(MODEL_ID)

    # Tokenização vetorizada com padding e truncamento para uniformizar o batch
    inputs = tok(
        sentences,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=256
    ).to("cuda", non_blocking=True)

    # Mede tempo de inferência com sincronização antes e depois
    torch.cuda.synchronize()
    t0 = time.perf_counter()

    # Geração paralela das traduções via beam search (melhor qualidade)
    outputs = model.generate(
        **inputs,
        max_new_tokens=MAX_NEW,
        num_beams=4,
        length_penalty=1.0,
        early_stopping=True
    )

    torch.cuda.synchronize()
    elapsed = time.perf_counter() - t0

    # Decodifica as saídas para texto em português
    pt_texts = tok.batch_decode(outputs, skip_special_tokens=True)

    # Impressão dos resultados
    print(f"\n=== mT5-base EN→PT • tradução batelada (GPU) — {elapsed:.2f} s ===\n")
    for en, pt in zip(sentences, pt_texts):
        print(f"EN: {en}\nPT: {pt}\n")

# ------------------------------------------------------------------------------
# PONTO DE ENTRADA
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    translate_batch(EN_SENTENCES)

/databricks/python/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(



=== mT5-base EN→PT • tradução batelada (GPU) — 0.24 s ===

EN: The XXV ERAD/RS 2025 will be held in Foz do Iguacu between April 23 and 25.
PT: true

EN: Participants can visit the Itaipu Parquetec facilities during the event.
PT: true

EN: The program features advanced courses on GPU programming and exascale computing.
PT: true

EN: The deadline for paper submissions to the Scientific Initiation Forum was extended again.
PT: false

EN: Winners of the CUDA kernel hackathon will receive cash prizes and free registration.
PT: true



---------------------------------------------------------------------------
OutOfMemoryError                          Traceback (most recent call last)
File <command-5741184051847171>, line 90
     86 # ------------------------------------------------------------------------------
     87 # PONTO DE ENTRADA
     88 # ------------------------------------------------------------------------------
     89 if __name__ == "__main__":
---> 90     translate_batch(EN_SENTENCES)

File /databricks/python/lib/python3.12/site-packages/torch/utils/_contextlib.py:116, in context_decorator.<locals>.decorate_context(*args, **kwargs)
    113 @functools.wraps(func)
    114 def decorate_context(*args, **kwargs):
    115     with ctx_factory():
--> 116         return func(*args, **kwargs)

File <command-5741184051847171>, line 51, in translate_batch(sentences)
     45 @torch.inference_mode()
     46 def translate_batch(sentences: list[str]) -> None:
     47     """
     48     Executa tradução de todas as

In [0]:
import os
import time
import random
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from typing import List, Dict

# ------------------------------------------------------------------------------
# CONFIGURAÇÕES DO EXPERIMENTO
# ------------------------------------------------------------------------------
MODEL_ID   = "unicamp-dl/ptt5-base-pt-msmarco-10k-v2"
CACHE_DIR  = os.path.expanduser("~/.cache/huggingface/transformers")
MAX_NEW    = 64                     # Tokens máximos por tradução
SIZES      = [2**i for i in range(5)]  # N de sentenças: 1 a 32
SEED       = 42
random.seed(SEED)

# ------------------------------------------------------------------------------
def load_model(model_id: str, cache_dir: str):
    """
    Baixa e carrega o modelo + tokenizer com cache local.
    Não faz distinção de CPU ou GPU aqui — isso é feito durante a execução.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_id, cache_dir=cache_dir)
    return tokenizer, model

tokenizer, model = load_model(MODEL_ID, CACHE_DIR)
model.eval()

# ------------------------------------------------------------------------------
def make_sentences(n: int) -> List[str]:
    """
    Gera N sentenças únicas em inglês para simular um cenário de tradução real.
    Isso evita reuso de tokens e força o modelo a processar cada entrada.
    """
    templates = [
        "Machine translation is a fundamental task in NLP, sample {}.",
        "This paper explores the role of deep learning in language processing, example {}.",
        "Translation models must handle linguistic variation, instance {}.",
        "Modern neural models outperform traditional methods, item {}.",
        "Language pairs with less data pose unique challenges, case {}."
    ]
    return [random.choice(templates).format(i) for i in range(n)]

# ------------------------------------------------------------------------------
@torch.inference_mode()
def run_cpu_seq(sentences: List[str]) -> float:
    """
    Traduz sequencialmente cada sentença na CPU.
    Este método não se beneficia de paralelismo, sendo útil como baseline.
    """
    model.to("cpu")
    t0 = time.perf_counter()
    for sentence in sentences:
        inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True).to("cpu")
        _ = model.generate(
            **inputs,
            max_new_tokens=MAX_NEW,
            num_beams=4,
            early_stopping=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            max_length=64
        )
    return time.perf_counter() - t0

# ------------------------------------------------------------------------------
@torch.inference_mode()
def run_gpu_batch(sentences: List[str]) -> float | None:
    """
    Traduz todas as sentenças de uma vez na GPU.
    Esta abordagem explora o paralelismo natural das GPUs via execução batelada.
    """
    if not torch.cuda.is_available():
        return None
    try:
        model.to("cuda", non_blocking=True)
        inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True).to("cuda", non_blocking=True)

        torch.cuda.synchronize()  # sincronia antes da medição
        t0 = time.perf_counter()

        _ = model.generate(
            **inputs,
            max_new_tokens=MAX_NEW,
            num_beams=4,
            early_stopping=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

        torch.cuda.synchronize()  # sincronia após geração
        return time.perf_counter() - t0
    finally:
        torch.cuda.empty_cache()

# ------------------------------------------------------------------------------
def benchmark() -> Dict[int, Dict[str, float | None]]:
    """
    Executa o benchmark para diferentes tamanhos de batch.
    Para cada N, mede tempo total na CPU (sequencial) e GPU (paralelo),
    calculando o speed-up obtido com computação de alto desempenho.
    """
    results: Dict[int, Dict[str, float | None]] = {}
    print(f"\n{'N':>5} | {'CPU-SEQ (s)':>11} | {'GPU (s)':>8} | Speed-up")
    print("-" * 43)

    for n in SIZES:
        prompts = make_sentences(n)

        t_cpu = run_cpu_seq(prompts)
        t_gpu = run_gpu_batch(prompts)

        speed = (t_cpu / t_gpu) if (t_gpu and t_gpu > 0) else None
        results[n] = {"cpu": t_cpu, "gpu": t_gpu, "speedup": speed}

        gpu_txt = f"{t_gpu:8.2f}" if t_gpu else "   N/A  "
        spd_txt = f"{speed:6.2f}×" if speed else "  —"
        print(f"{n:5d} | {t_cpu:11.2f} | {gpu_txt} | {spd_txt}")

    return results

# ------------------------------------------------------------------------------
if __name__ == "__main__":
    benchmark()

2025-04-25 11:25:34.449262: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-25 11:25:34.465378: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-25 11:25:34.485401: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-25 11:25:34.491454: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-25 11:25:34.506083: I tensorflow/core/platform/cpu_feature_guar

tokenizer_config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


...

Both `max_new_tokens` (=64) and `max_length`(=64) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=64) and `max_length`(=64) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=64) and `max_length`(=64) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=64) and `max_length`(=64) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `ma

    8 |        0.55 |     0.06 |   8.98×


Both `max_new_tokens` (=64) and `max_length`(=64) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=64) and `max_length`(=64) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=64) and `max_length`(=64) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=64) and `max_length`(=64) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `ma

   16 |        1.13 |     0.06 |  17.68×


---------------------------------------------------------------------------
OutOfMemoryError                          Traceback (most recent call last)
File <command-5741184051847141>, line 126
    124 # ------------------------------------------------------------------------------
    125 if __name__ == "__main__":
--> 126     benchmark()

File <command-5741184051847141>, line 113, in benchmark()
    110 prompts = make_sentences(n)
    112 t_cpu = run_cpu_seq(prompts)
--> 113 t_gpu = run_gpu_batch(prompts)
    115 speed = (t_cpu / t_gpu) if (t_gpu and t_gpu > 0) else None
    116 results[n] = {"cpu": t_cpu, "gpu": t_gpu, "speedup": speed}

File /databricks/python/lib/python3.12/site-packages/torch/utils/_contextlib.py:116, in context_decorator.<locals>.decorate_context(*args, **kwargs)
    113 @functools.wraps(func)
    114 def decorate_context(*args, **kwargs):
    115     with ctx_factory():
--> 116         return func(*args, **kwargs)

File <command-5741184051847141>, line 78, in r

#5 - Perguntas e Respostas em Português com BERT

📌 Resumo do Exercício

Este exercício realiza inferência batelada (em lote) de perguntas-e-respostas (QA) com um modelo BERT treinado em português. Os pares (contexto, pergunta) são processados simultaneamente na GPU, e o modelo retorna o trecho exato que responde à pergunta, com base em logits de início e fim dos tokens.

💡 Aplicação

Aplicações práticas incluem:
- Sistemas de FAQ automáticos em português
- Chatbots com compreensão de contexto
- Sistemas de busca por resposta exata (ex: jurídico, educacional)
- Aceleração de pipelines de QA em ambientes de produção

🔁 Input e Output

- Input: Lista de pares (contexto, pergunta) em português.
- Output: Lista de respostas extraídas diretamente dos contextos — respostas curtas, objetivas, com base textual (span-based QA).

In [0]:
from __future__ import annotations
import os, time, torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# ------------------------------------------------------------------------------
# CONFIGURAÇÃO DO MODELO E CACHE
# ------------------------------------------------------------------------------
MODEL_ID  = "pierreguillou/bert-base-cased-squad-v1.1-portuguese"
CACHE_DIR = os.path.expanduser("~/.cache/huggingface/transformers")

# ------------------------------------------------------------------------------
# DADOS DE ENTRADA (Contextos reais da ERAD/RS + perguntas correspondentes)
# ------------------------------------------------------------------------------
QA_PAIRS = [
    (
        "A XXV Escola Regional de Alto Desempenho da Região Sul (ERAD/RS 2025) "
        "acontecerá de 23 a 25 de abril em Foz do Iguaçu, no Paraná. O evento "
        "reúne minicursos, palestras e uma visita técnica ao Itaipu Parquetec.",
        "Em que cidade será realizada a ERAD/RS 2025?"
    ),
    (
        "O Comitê de Programa da ERAD/RS prorrogou o prazo de submissão de "
        "trabalhos para o Fórum de Iniciação Científica. Os artigos podem ter "
        "até oito páginas e devem abordar temas de computação de alto desempenho.",
        "Qual foi o prazo prorrogado pela organização?"
    ),
    (
        "Durante a ERAD/RS 2025 haverá um hackathon focado em otimização de "
        "kernels CUDA, utilizando clusters com GPUs NVIDIA A100. As melhores "
        "equipes receberão prêmios em dinheiro.",
        "Qual tecnologia de GPU será usada no hackathon?"
    ),
]

# ------------------------------------------------------------------------------
# FUNÇÃO DE CARREGAMENTO DO MODELO
# ------------------------------------------------------------------------------
def load_model(model_id: str):
    """
    Carrega tokenizer e modelo diretamente para GPU.
    Utiliza FP16 para otimizar memória e acelerar a inferência.
    """
    if not torch.cuda.is_available():
        raise RuntimeError("🚫 Esta versão exige GPU CUDA — nenhuma placa detectada.")

    tok = AutoTokenizer.from_pretrained(model_id, cache_dir=CACHE_DIR)
    model = AutoModelForQuestionAnswering.from_pretrained(
        model_id,
        cache_dir=CACHE_DIR,
        torch_dtype=torch.float16  # usa meia precisão para maior eficiência
    ).to("cuda")

    return tok, model

# ------------------------------------------------------------------------------
# INFERÊNCIA EM LOTE NA GPU
# ------------------------------------------------------------------------------
@torch.inference_mode()
def answer_batch(pairs):
    """
    Executa QA em lote, aproveitando o paralelismo da GPU.
    Cada par (contexto, pergunta) é processado simultaneamente.
    """
    tok, model = load_model(MODEL_ID)

    # Separação dos pares em listas de perguntas e contextos
    questions, contexts = zip(*pairs)

    # Tokenização paralela e envio para a GPU
    inputs = tok(
        list(questions),
        list(contexts),
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to("cuda")

    # Medição de tempo com sincronização explícita
    torch.cuda.synchronize()
    t0 = time.perf_counter()
    outputs = model(**inputs)
    torch.cuda.synchronize()
    elapsed = time.perf_counter() - t0

    # Inicializa vetor de respostas
    answers = [""] * len(pairs)

    # Extração do trecho com maior pontuação entre os logits de início e fim
    for i in range(len(pairs)):
        start_idx = int(torch.argmax(outputs.start_logits[i]))
        end_idx   = int(torch.argmax(outputs.end_logits[i])) + 1
        tokens    = inputs["input_ids"][i][start_idx:end_idx]
        answers[i] = tok.decode(tokens, skip_special_tokens=True)

    # Impressão dos resultados
    print(f"\n=== BERT-SQuAD-PT • QA batelada (GPU) — {elapsed:.2f} s ===\n")
    for (ctx, q), ans in zip(pairs, answers):
        print(f"Pergunta : {q}\nResposta  : {ans}\n")

# ------------------------------------------------------------------------------
# PONTO DE ENTRADA
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    answer_batch(QA_PAIRS)

In [0]:
import os
import time
import random
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from typing import List, Dict, Tuple

# ------------------------------------------------------------------------------
# CONFIGURAÇÕES DO EXPERIMENTO
# ------------------------------------------------------------------------------
MODEL_ID   = "pierreguillou/bert-base-cased-squad-v1.1-portuguese"
CACHE_DIR  = os.path.expanduser("~/.cache/huggingface/transformers")
SIZES      = [2**i for i in range(9)]  # N de pares (1 até 256)
SEED       = 42
random.seed(SEED)

# ------------------------------------------------------------------------------
def load_model(model_id: str, cache_dir: str):
    """
    Carrega tokenizer e modelo da Hugging Face com cache local.
    A escolha do dispositivo (CPU/GPU) é feita posteriormente na execução.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)
    model = AutoModelForQuestionAnswering.from_pretrained(model_id, cache_dir=cache_dir)
    return tokenizer, model

tokenizer, model = load_model(MODEL_ID, CACHE_DIR)
model.eval()

# ------------------------------------------------------------------------------
def make_qa_pairs(n: int) -> List[Tuple[str, str]]:
    """
    Gera N pares de (contexto, pergunta) com variações na pergunta.
    O contexto é sempre o mesmo para permitir comparação justa de performance.
    """
    base_context = (
        "A XXV Escola Regional de Alto Desempenho da Região Sul (ERAD/RS 2025) acontecerá de 23 a 25 de abril em Foz do Iguaçu, no Paraná. O evento reúne minicursos, palestras e uma visita técnica ao Itaipu Parquetec.",
        "O Comitê de Programa da ERAD/RS prorrogou o prazo de submissão de trabalhos para o Fórum de Iniciação Científica. Os artigos podem ter até oito páginas e devem abordar temas de computação de alto desempenho.",
        "Durante a ERAD/RS 2025 haverá um hackathon focado em otimização de kernels CUDA, utilizando clusters com GPUs NVIDIA A100. As melhores equipes receberão prêmios em dinheiro.",
    )
    questions = [
        "Qual tecnologia de GPU será usada no hackathon?",
        "Qual foi o prazo prorrogado pela organização?",
        "Em que cidade será realizada a ERAD/RS 2025?"
    ]
    return [(base_context, random.choice(questions)) for _ in range(n)]

# ------------------------------------------------------------------------------
@torch.inference_mode()
def run_cpu_seq(pairs: List[Tuple[str, str]]) -> float:
    """
    Executa inferência uma por uma na CPU.
    Não há paralelismo aqui: cada par é processado individualmente.
    """
    model.to("cpu")
    t0 = time.perf_counter()
    for context, question in pairs:
        inputs = tokenizer(question, context, return_tensors="pt", truncation=True, padding=True).to("cpu")
        _ = model(**inputs)
    return time.perf_counter() - t0

# ------------------------------------------------------------------------------
@torch.inference_mode()
def run_gpu_batch(pairs: List[Tuple[str, str]]) -> float | None:
    """
    Executa inferência de todos os pares em um único batch na GPU.
    A tokenização e a execução são paralelizáveis e otimizadas pela GPU.
    """
    if not torch.cuda.is_available():
        return None

    try:
        model.to("cuda", non_blocking=True)
        questions, contexts = zip(*pairs)
        inputs = tokenizer(
            list(questions), list(contexts),
            return_tensors="pt",
            padding=True, truncation=True
        ).to("cuda", non_blocking=True)

        torch.cuda.synchronize()  # sincroniza antes da medição
        t0 = time.perf_counter()
        _ = model(**inputs)
        torch.cuda.synchronize()  # sincroniza após execução
        return time.perf_counter() - t0

    finally:
        del inputs
        torch.cuda.empty_cache()

# ------------------------------------------------------------------------------
def benchmark() -> Dict[int, Dict[str, float | None]]:
    """
    Executa o benchmark variando o número de pares QA.
    Para cada N, registra tempos absolutos e ganho de speed-up.
    """
    results: Dict[int, Dict[str, float | None]] = {}
    print(f"\n{'N':>5} | {'CPU-SEQ (s)':>11} | {'GPU (s)':>8} | Speed-up")
    print("-" * 43)

    for n in SIZES:
        qa_pairs = make_qa_pairs(n)

        t_cpu = run_cpu_seq(qa_pairs)
        t_gpu = run_gpu_batch(qa_pairs)

        speed = (t_cpu / t_gpu) if (t_gpu and t_gpu > 0) else None
        results[n] = {"cpu": t_cpu, "gpu": t_gpu, "speedup": speed}

        gpu_txt = f"{t_gpu:8.2f}" if t_gpu else "   N/A  "
        spd_txt = f"{speed:6.2f}×" if speed else "  —"
        print(f"{n:5d} | {t_cpu:11.2f} | {gpu_txt} | {spd_txt}")

    return results

# ------------------------------------------------------------------------------
if __name__ == "__main__":
    benchmark()

#6 - Reconhecimento de Entidades Nomeadas com BERTimbau-Base


📌 Resumo do Exercício

Este exercício demonstra como realizar Reconhecimento de Entidades Nomeadas (NER) em português brasileiro, utilizando o modelo marcosgg/bert-base-pt-ner-enamex, baseado no BERTimbau. O modelo é aplicado sobre várias sentenças de forma paralela, usando inferência em lote na GPU, e o tempo de execução é medido com precisão.

💡 Aplicação
O NER é amplamente utilizado em:
- Extração de nomes de pessoas, organizações, locais em textos jornalísticos e científicos
- Indexação semântica em mecanismos de busca
- Automatização de análise jurídica e contratos
- Análise de currículos e documentos administrativos
- Pré-processamento em sistemas de recomendação e chatbots

🔁 Input e Output
- Input: Lista de sentenças em português contendo possíveis entidades nomeadas.
- Output: Para cada sentença, retorna as entidades encontradas, seu tipo (ex: PER, LOC, ORG) e a confiança da predição.

In [0]:
# ==============================================================================
# IMPORTAÇÃO DE BIBLIOTECAS
# ==============================================================================
import os
import time
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TokenClassificationPipeline
)

# ==============================================================================
# CONFIGURAÇÃO DO MODELO E EXEMPLOS DE TESTE
# ==============================================================================
MODEL_ID  = "marcosgg/bert-base-pt-ner-enamex"  # Modelo treinado para português (ENAMEX)
CACHE_DIR = os.path.expanduser("~/.cache/huggingface/transformers")

SENTENCES = [
    "A XXV ERAD/RS 2025 ocorrerá em Foz do Iguaçu, no Paraná, entre 23 e 25 de abril.",
    "Os participantes visitarão o Itaipu Parquetec durante o evento.",
    "Haverá minicursos sobre exaescala com instrutores da Universidade Federal do Rio Grande do Sul.",
    "O hackathon de otimização de kernels CUDA contará com apoio da NVIDIA.",
    "Trabalhos aceitos no Fórum de Iniciação Científica serão publicados pela SBC."
]

# ==============================================================================
# FUNÇÃO: CARREGAMENTO DO MODELO NA GPU
# ==============================================================================
def load_pipeline(model_id: str) -> TokenClassificationPipeline:
    """
    Carrega o modelo e tokenizer, e prepara uma pipeline para classificação de tokens.
    Atribuição explícita à GPU com dtype otimizado (FP16) para economizar memória da T4.
    """
    if not torch.cuda.is_available():
        raise RuntimeError("🚫 Esta versão requer uma GPU CUDA; nenhuma detectada.")

    tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=CACHE_DIR)
    model = AutoModelForTokenClassification.from_pretrained(
        model_id,
        cache_dir=CACHE_DIR,
        torch_dtype=torch.float16  # Reduz uso de VRAM mantendo performance
    ).to("cuda")

    return TokenClassificationPipeline(
        model=model,
        tokenizer=tokenizer,
        device=0,  # 0 corresponde à GPU atual
        aggregation_strategy="simple"  # Agrupa tokens contíguos da mesma entidade
    )

# ==============================================================================
# FUNÇÃO: EXECUÇÃO DO RECONHECIMENTO EM LOTE
# ==============================================================================
@torch.inference_mode()
def ner_batch(sentences: list[str]) -> None:
    """
    Executa a inferência do modelo NER para um lote de frases, usando GPU.
    A função mede o tempo total de execução e imprime entidades detectadas.
    """
    pipe = load_pipeline(MODEL_ID)

    # Sincronização de CUDA antes da medição para eliminar operações pendentes
    torch.cuda.synchronize()
    t0 = time.perf_counter()

    # Aqui ocorre o paralelismo: todas as frases são processadas em lote
    results = pipe(sentences, batch_size=len(sentences), truncation=True)

    # Sincronização após execução para medir corretamente o tempo de inferência
    torch.cuda.synchronize()
    elapsed = time.perf_counter() - t0

    print(f"\n=== BERTimbau NER • lote único (GPU) — {elapsed:.2f} s ===\n")
    for sent, ents in zip(sentences, results):
        print(f"🔹 Sentença: {sent}")
        for ent in ents:
            print(f"   ↳ {ent['word']}  |  {ent['entity_group']}  |  score {ent['score']:.2f}")
        print()

# ==============================================================================
# EXECUÇÃO PRINCIPAL
# ==============================================================================
if __name__ == "__main__":
    ner_batch(SENTENCES)


In [0]:
# ==============================================================================
# IMPORTAÇÃO DE BIBLIOTECAS
# ==============================================================================
import os
import time
import random
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
from typing import List, Dict

# ==============================================================================
# CONFIGURAÇÃO DO MODELO
# ==============================================================================
MODEL_ID   = "marcosgg/bert-base-pt-ner-enamex"
CACHE_DIR  = os.path.expanduser("~/.cache/huggingface/transformers")
SIZES      = [2**i for i in range(9)]  # Número de sentenças: 1 a 256
SEED       = 42
random.seed(SEED)

# ==============================================================================
# FUNÇÃO DE CARREGAMENTO DO MODELO
# ==============================================================================
def load_model(model_id: str, cache_dir: str):
    tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)
    model = AutoModelForTokenClassification.from_pretrained(model_id, cache_dir=cache_dir)
    return tokenizer, model

tokenizer, model = load_model(MODEL_ID, CACHE_DIR)
model.eval()

# ==============================================================================
# GERAÇÃO DE SENTENÇAS EXEMPLO
# ==============================================================================
def make_sentences(n: int) -> List[str]:
    """Gera sentenças contendo entidades nomeadas (NER)."""
    templates = [
    "A XXV ERAD/RS 2025 ocorrerá em Foz do Iguaçu, no Paraná, entre 23 e 25 de abril.",
    "Os participantes visitarão o Itaipu Parquetec durante o evento.",
    "Haverá minicursos sobre exaescala com instrutores da Universidade Federal do Rio Grande do Sul.",
    "O hackathon de otimização de kernels CUDA contará com apoio da NVIDIA.",
    "Trabalhos aceitos no Fórum de Iniciação Científica serão publicados pela SBC."
    ]
    return [random.choice(templates) + f" Example {i}" for i in range(n)]

# ==============================================================================
# INFERÊNCIA SEQUENCIAL NA CPU
# ==============================================================================
@torch.inference_mode()
def run_cpu_seq(sentences: List[str]) -> float:
    """
    Executa NER uma frase por vez usando CPU. Simula casos onde não há paralelismo.
    Ideal para entender a latência por amostra em ambientes de baixa capacidade.
    """
    model.to("cpu")
    t0 = time.perf_counter()
    for sent in sentences:
        inputs = tokenizer(sent, return_tensors="pt", truncation=True, padding=True).to("cpu")
        _ = model(**inputs)
    return time.perf_counter() - t0

# ==============================================================================
# INFERÊNCIA EM LOTE NA GPU
# ==============================================================================
@torch.inference_mode()
def run_gpu_batch(sentences: List[str]) -> float | None:
    """
    Executa NER sobre todas as frases de uma só vez (paralelismo em batch).
    É aqui que a GPU brilha: processando várias entradas em paralelo com grande throughput.
    """
    if not torch.cuda.is_available():
        return None
    try:
        model.to("cuda", non_blocking=True)
        inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True).to("cuda", non_blocking=True)
        torch.cuda.synchronize()
        t0 = time.perf_counter()
        _ = model(**inputs)
        torch.cuda.synchronize()
        return time.perf_counter() - t0
    finally:
        del inputs
        torch.cuda.empty_cache()

# ==============================================================================
# FUNÇÃO DE BENCHMARK COMPARATIVO
# ==============================================================================
def benchmark() -> Dict[int, Dict[str, float | None]]:
    """
    Executa benchmark comparando os tempos de execução entre CPU e GPU
    para diferentes quantidades de sentenças.
    """
    results: Dict[int, Dict[str, float | None]] = {}
    print(f"\n{'N':>5} | {'CPU-SEQ (s)':>11} | {'GPU (s)':>8} | Speed-up")
    print("-" * 43)

    for n in SIZES:
        sentences = make_sentences(n)

        t_cpu = run_cpu_seq(sentences)
        t_gpu = run_gpu_batch(sentences)

        speed = (t_cpu / t_gpu) if (t_gpu and t_gpu > 0) else None
        results[n] = {"cpu": t_cpu, "gpu": t_gpu, "speedup": speed}

        gpu_txt = f"{t_gpu:8.2f}" if t_gpu else "   N/A  "
        spd_txt = f"{speed:6.2f}×" if speed else "  —"
        print(f"{n:5d} | {t_cpu:11.2f} | {gpu_txt} | {spd_txt}")

    return results

# ==============================================================================
# EXECUÇÃO PRINCIPAL
# ==============================================================================
if __name__ == "__main__":
    benchmark()


#7 - Fine Tuning de BERTimbau para Classificação Binária de Sentimentos



📌 Resumo do Exercício

Este exercício apresenta o processo completo de fine-tuning (ajuste fino) de um modelo de linguagem pré-treinado, o BERTimbau, para uma tarefa supervisionada de classificação binária de sentimentos. O treinamento é executado em lote na GPU, utilizando o Trainer da Hugging Face. A base de dados é sintética, com frases positivas e negativas simulando avaliações sobre o evento ERAD/RS 2025.

💡 Aplicação

Esse tipo de ajuste fino é usado em:
- Classificadores de sentimento para redes sociais, SAC, e-commerce etc.
- Customização de modelos genéricos para domínios específicos (jurídico, educacional, técnico).
- Aprimoramento de chatbots com respostas emocionais mais contextuais.
- Treinamento supervisionado de classificadores especializados usando poucas amostras.

🔁 Input e Output

- Input: Dataset sintético contendo frases com sentimentos positivos e negativos (em português), com rótulo 1 para positivo e 0 para negativo.
- Output: Modelo ajustado para distinguir o sentimento das frases. A saída principal deste script é o tempo de treinamento com batch e GPU, útil para benchmarks.

In [0]:
# ==============================================================================
# IMPORTAÇÃO DE BIBLIOTECAS
# ==============================================================================
from __future__ import annotations
import os, time, random, torch, tempfile, shutil
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    Trainer, TrainingArguments
)
from datasets import Dataset

# ==============================================================================
# CONFIGURAÇÃO DO MODELO E AMBIENTE
# ==============================================================================
MODEL_ID  = "neuralmind/bert-base-portuguese-cased"
CACHE_DIR = os.path.expanduser("~/.cache/huggingface/transformers")
SEED      = 42
random.seed(SEED)
torch.manual_seed(SEED)

# ==============================================================================
# GERAÇÃO DO DATASET SINTÉTICO BALANCEADO
# ==============================================================================
def gerar_dataset(n: int = 600) -> Dataset:
    """Cria um dataset balanceado de frases positivas e negativas sobre a ERAD/RS."""
    pos_base = [
        "Estou ansioso para participar da ERAD/RS 2025 em Foz do Iguaçu!",
        "A visita técnica à Itaipu Parquetec será fantástica.",
        "Os minicursos de exaescala parecem imperdíveis.",
        "Adorei a prorrogação do prazo, terei tempo de submeter meu artigo.",
        "O hackathon com GPUs NVIDIA vai ser incrível."
    ]
    neg_base = [
        "Achei a taxa de inscrição da ERAD/RS muito alta.",
        "Fiquei frustrado com o atraso na divulgação dos tutoriais.",
        "Não gostei que o evento não oferece opções de hospedagem acessíveis.",
        "A mudança de cronograma me atrapalhou bastante.",
        "Achei a página de submissão confusa."
    ]
    textos, labels = [], []
    for i in range(n):
        if i % 2 == 0:
            textos.append(random.choice(pos_base) + f" (exemplo {i})")
            labels.append(1)
        else:
            textos.append(random.choice(neg_base) + f" (exemplo {i})")
            labels.append(0)
    return Dataset.from_dict({"text": textos, "label": labels})

# ==============================================================================
# TOKENIZAÇÃO COM PADDING/TRUNCATION
# ==============================================================================
def tokenize(ds: Dataset, tok) -> Dataset:
    return ds.map(
        lambda x: tok(x["text"], truncation=True, padding="max_length"),
        batched=True
    )

# ==============================================================================
# FINE-TUNING NA GPU — ATIVAÇÃO DE PARALELISMO
# ==============================================================================
@torch.inference_mode(False)
def treinar_gpu(dataset: Dataset) -> float:
    """
    Realiza treinamento em GPU com suporte a FP16.
    Importante: toda a carga do modelo e dados são transferidos para CUDA,
    ativando paralelismo massivo no treinamento com batch size 16.
    """
    if not torch.cuda.is_available():
        raise RuntimeError("🚫 Nenhuma GPU CUDA detectada – esta versão exige GPU.")

    # Carrega tokenizer e transforma o dataset para o formato compatível
    tok = AutoTokenizer.from_pretrained(MODEL_ID, cache_dir=CACHE_DIR)
    ds_tok = tokenize(dataset, tok).remove_columns(["text"])
    ds_tok.set_format("torch")

    # Carrega o modelo em formato FP16 direto na GPU
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_ID,
        cache_dir=CACHE_DIR,
        num_labels=2,
        torch_dtype=torch.float16
    ).to("cuda")

    # Cria diretório temporário para não salvar nada em disco
    workdir = tempfile.mkdtemp(prefix="eradrs_finetune_")

    # Configurações do HuggingFace Trainer
    args = TrainingArguments(
        output_dir=workdir,
        num_train_epochs=1,
        per_device_train_batch_size=16,
        learning_rate=2e-5,
        logging_steps=20,
        seed=SEED,
        report_to="none",
        save_strategy="no"  # não salva checkpoints
    )

    trainer = Trainer(model=model, args=args, train_dataset=ds_tok)

    # Início da contagem com sincronização CUDA (para medir com precisão)
    torch.cuda.synchronize()
    t0 = time.perf_counter()
    trainer.train()
    torch.cuda.synchronize()
    elapsed = time.perf_counter() - t0

    shutil.rmtree(workdir, ignore_errors=True)
    return elapsed

# ==============================================================================
# EXECUÇÃO PRINCIPAL
# ==============================================================================
if __name__ == "__main__":
    ds = gerar_dataset(n=600)
    print("\nFine-tuning BERTimbau (1 época, 600 exemplos)")
    print(f"Máquina: g4dn.8xlarge (T4 16 GB)  ·  Batch 16  ·  FP16\n")
    tempo = treinar_gpu(ds)
    print(f"Tempo total de treinamento na GPU: {tempo:.2f} s")


In [0]:
# ==============================================================================
# IMPORTAÇÃO DE BIBLIOTECAS
# ==============================================================================
import os
import time
import random
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
from datasets import Dataset
from typing import List

# ==============================================================================
# CONFIGURAÇÃO GLOBAL
# ==============================================================================
MODEL_ID = "neuralmind/bert-base-portuguese-cased"
CACHE_DIR = os.path.expanduser("~/.cache/huggingface/transformers")
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)

# ==============================================================================
# GERAÇÃO DO DATASET SINTÉTICO
# ==============================================================================
def generate_data(n: int = 1000) -> Dataset:
    """
    Cria um dataset de sentimentos simulados com exemplos balanceados.
    0 = negativo, 1 = positivo.
    """
    texts = [
        f"Estou ansioso para participar da ERAD/RS 2025 em Foz do Iguaçu!, exemplo {i}" if i % 2 == 0
        else f"Achei a página de submissão confusa., exemplo {i}"
        for i in range(n)
    ]
    labels = [1 if i % 2 == 0 else 0 for i in range(n)]
    return Dataset.from_dict({"text": texts, "label": labels})

# ==============================================================================
# TOKENIZAÇÃO
# ==============================================================================
def tokenize_dataset(ds: Dataset, tokenizer) -> Dataset:
    """
    Transforma textos em tensores usando o tokenizer DistilBERT.
    Inclui truncamento e padding fixo (necessário para batching).
    """
    return ds.map(lambda x: tokenizer(x["text"], truncation=True, padding="max_length"), batched=True)

# ==============================================================================
# FASE DE TREINAMENTO (CPU OU GPU)
# ==============================================================================
def train_model(dataset: Dataset, device: str = "cpu") -> float:
    """
    Realiza treinamento por 1 época em CPU ou GPU.
    Retorna o tempo total (em segundos).
    """
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, cache_dir=CACHE_DIR)
    tokenized = tokenize_dataset(dataset, tokenizer).remove_columns(["text"])
    tokenized.set_format("torch")

    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_ID, cache_dir=CACHE_DIR, num_labels=2, ignore_mismatched_sizes=True
    )

    output_dir = f"/tmp/tmp_finetune_{device}"
    training_args = TrainingArguments(
        output_dir=output_dir,
        eval_strategy="no",
        per_device_train_batch_size=8,
        num_train_epochs=1,
        logging_steps=50,
        seed=SEED,
        report_to="none",
        save_strategy="no",
        save_steps=0,
        no_cuda=(device == "cpu")  # importante: ativa ou desativa GPU
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized,
    )

    # Início da medição (com sincronização se for GPU)
    if device == "cuda":
        torch.cuda.synchronize()
    t0 = time.perf_counter()
    trainer.train()
    if device == "cuda":
        torch.cuda.synchronize()
    return time.perf_counter() - t0

# ==============================================================================
# BENCHMARK COMPARATIVO
# ==============================================================================
def benchmark():
    dataset = generate_data(n=500)  # dataset leve para demonstração rápida
    print("\nBenchmark: Fine-tuning DistilBERT for 1 epoch on 500 examples")
    print(f"{'Device':>6} | {'Time (s)':>9}")
    print("-" * 21)

    # Execução sequencial em CPU (sem paralelismo massivo)
    t_cpu = train_model(dataset, device="cpu")
    print(f"{'CPU':>6} | {t_cpu:9.2f}")

    # Execução paralela na GPU (ativando CUDA e uso massivo de tensores FP32)
    if torch.cuda.is_available():
        t_gpu = train_model(dataset, device="cuda")
        print(f"{'GPU':>6} | {t_gpu:9.2f}")
        print(f"\nSpeed-up: {t_cpu / t_gpu:.2f}×")
    else:
        print(f"{'GPU':>6} | {'  N/A':>9}")

# ==============================================================================
# EXECUÇÃO PRINCIPAL
# ==============================================================================
if __name__ == "__main__":
    benchmark()


#8 - Preenchimento de Máscara com BERTimbau (Fill-Mask)

📌 Resumo do Exercício

Este exercício demonstra como aplicar o modelo BERTimbau para a tarefa de Masked Language Modeling (MLM) — isto é, prever palavras ausentes em frases contendo o token [MASK]. A execução ocorre de forma paralela em lote na GPU, e o tempo de inferência é medido para diferentes volumes de entrada (de 32 a 2048 frases). O objetivo principal é evidenciar o impacto positivo do paralelismo vetorizado com CUDA em tarefas de inferência textual com BERT.

💡 Aplicação

Modelos MLM são usados em:
- Pré-treinamento de modelos BERT-like;
- Ferramentas de autocorreção ou sugestão de palavras;
- Recuperação de informação com base em contexto parcial;
- Aprendizado contextual de palavras em tarefas downstream;
- Geração de datasets com data augmentation por substituição de tokens.

🔁 Input e Output

- Input: Lista de frases em português com um token [MASK] por sentença, como: "O hackathon usará GPUs [MASK]. Amostra 42."
- Output: O modelo realiza uma passagem direta (forward pass), gerando logits para prever qual palavra deve ocupar o lugar do token [MASK]. Neste benchmark, o output analisado é o tempo de execução, e não a predição textual em si.

In [0]:
from __future__ import annotations
import os, time, random, torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
from typing import List

# ==============================================================================
# CONFIGURAÇÃO
# ==============================================================================
MODEL_ID   = "neuralmind/bert-base-portuguese-cased"  # 🇧🇷 BERTimbau-Base
CACHE_DIR  = os.path.expanduser("~/.cache/huggingface/transformers")
MASK_TOKEN = "[MASK]"
SIZES      = [2**i for i in range(5, 12)]   # de 32 até 2048 sentenças
SEED       = 42
random.seed(SEED); torch.manual_seed(SEED)

# ==============================================================================
# CARREGAMENTO DO MODELO + TOKENIZER (em FP16 direto na GPU)
# ==============================================================================
def load_model(model_id: str):
    if not torch.cuda.is_available():
        raise RuntimeError("🚫 Esta versão é apenas-GPU — nenhuma CUDA detectada.")

    # Tokenizer carrega vocabulário e regras de segmentação
    tok = AutoTokenizer.from_pretrained(model_id, cache_dir=CACHE_DIR)

    # Modelo de linguagem mascarada (BERT) carregado em float16 para otimizar VRAM
    mdl = AutoModelForMaskedLM.from_pretrained(
        model_id,
        cache_dir=CACHE_DIR,
        torch_dtype=torch.float16
    ).to("cuda")  # envia tudo para a GPU
    return tok, mdl

tokenizer, model = load_model(MODEL_ID)
model.eval()  # importante: desativa dropout, ativa modo de inferência

# ==============================================================================
# GERAÇÃO DE SENTENÇAS COM MÁSCARA (exemplos baseados na ERAD/RS)
# ==============================================================================
def make_masked_sentences(n: int) -> List[str]:
    """Cria *n* frases com token [MASK] para inferência."""
    templates = [
        f"A ERAD/RS 2025 acontecerá em {MASK_TOKEN}, no Paraná. Exemplo {{}}.",
        f"Os minicursos abordarão computação de {MASK_TOKEN} escala. Caso {{}}.",
        f"O hackathon de otimização usará GPUs {MASK_TOKEN}. Amostra {{}}.",
        f"A visita técnica ocorrerá no {MASK_TOKEN} Parquetec. Variante {{}}.",
        f"O prazo de submissão foi {MASK_TOKEN} pela organização. Item {{}}."
    ]
    return [random.choice(templates).format(i) for i in range(n)]

# ==============================================================================
# EXECUÇÃO EM LOTE NA GPU
# ==============================================================================
@torch.inference_mode()
def run_gpu_batch(sentences: List[str]) -> float:
    """
    Realiza inferência do tipo fill-mask sobre todas as sentenças em lote na GPU.
    Retorna o tempo total decorrido.
    """
    # Pré-processamento: tokenização + tensorização → CUDA
    inputs = tokenizer(
        sentences,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=64
    ).to("cuda", non_blocking=True)

    # Medição sincronizada do tempo de execução na GPU
    torch.cuda.synchronize(); t0 = time.perf_counter()
    _ = model(**inputs)  # forward pass com tudo vetorizado na T4
    torch.cuda.synchronize(); t1 = time.perf_counter()

    return t1 - t0

# ==============================================================================
# BENCHMARK COMPARATIVO: ESCALABILIDADE COM LOTE
# ==============================================================================
def benchmark():
    print("\nBenchmark Fill-Mask – BERTimbau (GPU, T4 16 GB)")
    print(f"{'N':>6} | {'GPU (s)':>8}")
    print("-" * 18)

    for n in SIZES:
        masked = make_masked_sentences(n)
        t_gpu  = run_gpu_batch(masked)
        print(f"{n:6d} | {t_gpu:8.2f}")

# ==============================================================================
# EXECUÇÃO PRINCIPAL
# ==============================================================================
if __name__ == "__main__":
    benchmark()


In [0]:
import os
import time
import random
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
from typing import List, Dict

# --------------------------- Configuração ---------------------------
MODEL_ID   = "neuralmind/bert-base-portuguese-cased"
CACHE_DIR  = os.path.expanduser("~/.cache/huggingface/transformers")
MASK_TOKEN = "[MASK]"
SIZES      = [2**i for i in range(5, 12)]  # de 32 até 2048 sentenças
SEED       = 42
random.seed(SEED)

# --------------------------- Carregamento ---------------------------
def load_model(model_id: str, cache_dir: str):
    tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)
    model = AutoModelForMaskedLM.from_pretrained(model_id, cache_dir=cache_dir, ignore_mismatched_sizes=True)
    return tokenizer, model

tokenizer, model = load_model(MODEL_ID, CACHE_DIR)
model.eval()  # modo inferência (desativa dropout etc.)

# --------------------------- Dados Sintéticos -----------------------
def make_masked_sentences(n: int) -> List[str]:
    """Gera *n* frases distintas com token [MASK]."""
    templates = [
        f"A ERAD/RS 2025 acontecerá em {MASK_TOKEN}, no Paraná. Exemplo {{}}.",
        f"Os minicursos abordarão computação de {MASK_TOKEN} escala. Caso {{}}.",
        f"O hackathon de otimização usará GPUs {MASK_TOKEN}. Amostra {{}}.",
        f"A visita técnica ocorrerá no {MASK_TOKEN} Parquetec. Variante {{}}.",
        f"O prazo de submissão foi {MASK_TOKEN} pela organização. Item {{}}."
    ]
    return [random.choice(templates).format(i) for i in range(n)]

# --------------------------- Inferência na CPU ----------------------
@torch.inference_mode()
def run_cpu_seq(sentences: List[str]) -> float:
    """
    Executa inferência de MLM sequencialmente na CPU.
    Cada sentença é processada isoladamente.
    """
    model.to("cpu")
    t0 = time.perf_counter()
    for sent in sentences:
        inputs = tokenizer(sent, return_tensors="pt", padding=True, truncation=True).to("cpu")
        _ = model(**inputs)
    return time.perf_counter() - t0

# --------------------------- Inferência na GPU ----------------------
@torch.inference_mode()
def run_gpu_batch(sentences: List[str]) -> float | None:
    """
    Executa inferência MLM em lote na GPU.
    Todas as sentenças são vetorizadas e processadas em paralelo.
    """
    if not torch.cuda.is_available():
        return None
    try:
        model.to("cuda", non_blocking=True)
        inputs = tokenizer(
            sentences,
            return_tensors="pt",
            padding=True,
            truncation=True
        ).to("cuda", non_blocking=True)

        torch.cuda.synchronize(); t0 = time.perf_counter()
        _ = model(**inputs)
        torch.cuda.synchronize()
        return time.perf_counter() - t0
    finally:
        del inputs
        torch.cuda.empty_cache()

# --------------------------- Benchmark Comparativo ------------------
def benchmark() -> Dict[int, Dict[str, float | None]]:
    """
    Executa benchmark variando o número de frases com [MASK].
    Compara tempo entre execução sequencial na CPU e execução em lote na GPU.
    """
    results: Dict[int, Dict[str, float | None]] = {}
    print(f"\n{'N':>5} | {'CPU-SEQ (s)':>11} | {'GPU (s)':>8} | Speed-up")
    print("-" * 43)

    for n in SIZES:
        masked = make_masked_sentences(n)
        t_cpu = run_cpu_seq(masked)
        t_gpu = run_gpu_batch(masked)
        speed = (t_cpu / t_gpu) if (t_gpu and t_gpu > 0) else None

        results[n] = {"cpu": t_cpu, "gpu": t_gpu, "speedup": speed}
        gpu_txt = f"{t_gpu:8.2f}" if t_gpu else "   N/A  "
        spd_txt = f"{speed:6.2f}×" if speed else "  —"
        print(f"{n:5d} | {t_cpu:11.2f} | {gpu_txt} | {spd_txt}")
    return results

# --------------------------- Execução Principal ---------------------
if __name__ == "__main__":
    benchmark()
